You have been hired by a rookie movie producer to help him decide what type of movies to produce and which actors to cast. You have to back your recommendations based on thorough analysis of the data he shared with you which has the list of 3000 movies and the corresponding details.
As a data scientist, you have to first explore the data and check its sanity.
Further, you have to answer the following questions:
### Which movie made the highest profit? Who were its producer and director? Identify the actors in that film.
### This data has information about movies made in different languages. Which language has the highest average ROI (return on investment)?
### Find out the unique genres of movies in this dataset.
### Make a table of all the producers and directors of each movie. Find the top 3 producers who have produced movies with the highest average RoI?
### Which actor has acted in the most number of movies? Deep dive into the movies, genres and profits corresponding to this actor.
### Top 3 directors prefer which actors the most?

In [3]:
#Import package

import pandas as pd
import numpy as np
 

path="/content/drive/MyDrive/DS/imdb_data.csv"


In [4]:
imdb_df = pd.read_csv(path)
imdb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   id                     3000 non-null   int64  
 1   belongs_to_collection  604 non-null    object 
 2   budget                 3000 non-null   int64  
 3   genres                 2993 non-null   object 
 4   homepage               946 non-null    object 
 5   imdb_id                3000 non-null   object 
 6   original_language      3000 non-null   object 
 7   original_title         3000 non-null   object 
 8   overview               2992 non-null   object 
 9   popularity             3000 non-null   float64
 10  poster_path            2999 non-null   object 
 11  production_companies   2844 non-null   object 
 12  production_countries   2945 non-null   object 
 13  release_date           3000 non-null   object 
 14  runtime                2998 non-null   float64
 15  spok

In [5]:
imdb_df.head()
imdb_df.loc[0,"crew"]
# converting string object type to actual list of dictionary using eval function

crew_list1= eval(imdb_df.loc[0,"crew"])
len(crew_list1)
imdb_df.info()
# keep only the columns needed
columns_to_keep= ["budget","genres","imdb_id","original_language","original_title","revenue","cast","crew"]
imdb_df = imdb_df[columns_to_keep]
imdb_df.info()
# drop all the N.A values

imdb_df_non_null = imdb_df.dropna(inplace= True)
# creating a function to convert string to list of dictionary

def convert_list_str(str):

  return eval(str)
  # null-values 
imdb_df[imdb_df["genres"].isna()]
# apply these functions to cast , crew and genres
imdb_df.loc[~imdb_df["genres"].isna(),"genres"]= imdb_df.loc[~imdb_df["genres"].isna(),"genres"].apply(convert_list_str)
imdb_df.loc[0,"genres"]
imdb_df.loc[~imdb_df["cast"].isna(),"cast"]= imdb_df.loc[~imdb_df["cast"].isna(),"cast"].apply(convert_list_str)
imdb_df.loc[0,"cast"]
imdb_df.loc[~imdb_df["crew"].isna(),"crew"]= imdb_df.loc[~imdb_df["crew"].isna(),"crew"].apply(convert_list_str)
imdb_df.loc[0,"crew"]
imdb_df_new = imdb_df.copy()
# we want to replace the low value of budget and revenue data with the median values

imdb_df_new.describe()
# The maximum value in budget in 380 million and min value is 0, similarly for revenue too the minimum and maximum are at two extremes.

# we replace all the data less than 1000 by corresponding median of the remaining data that are greater than 1000 for budget and revenue


imdb_df_new.loc[imdb_df_new["budget"]<1000,"budget"] = imdb_df_new.loc[imdb_df_new["budget"]>=1000,"budget"].median()
imdb_df_new.loc[imdb_df_new["revenue"]<1000,"revenue"] = imdb_df_new.loc[imdb_df_new["revenue"]>=1000,"revenue"].median()

imdb_df_new.describe()
# Now we can create a new column for profit=  Revenue-Budget

imdb_df_new["Profit"] = imdb_df_new["revenue"]- imdb_df_new["budget"]
# Finding the maximum profit made by a film

max_profit = imdb_df_new["Profit"].max()
max_profit
# Identifying the movie which made the highest profit and storing it as new dataframe

max_profit_df = imdb_df_new[imdb_df_new["Profit"]==max_profit]
max_profit_df
# Now identifying the cast, directors and producers of this highest grossing film

crew_list= max_profit_df.iloc[0]["crew"]
for item in crew_list:
  if item["job"] == "Director" :

    print(f"The director of the highest grossing film is {item['name']}")

  elif item["job"] == "Producer" :

    print(f"The Producer of the highest grossing film is {item['name']}")
    # getting the actors acted in the highest grossing movie


cast_list = max_profit_df.iloc[0]["cast"]
cast_list
actor_list= []

for item in cast_list:
  actor_list.append(item["name"])
  print(actor_list)
  actor_list_df = pd.DataFrame(actor_list)
  actor_list_df.set_axis(['Actors'], axis=1,inplace=True)
  actor_list_df
  # Calculate the ROI for the data by using the relation ROI= Profit/ Budget

imdb_df_new["ROI"]= imdb_df_new["Profit"]/imdb_df_new["budget"]
imdb_df_new.head()
# Groupby original language column w.r.t average of ROI column

roi_languages_df = imdb_df_new.groupby("original_language")["ROI"].mean().reset_index()
roi_languages_df
# Finding the maximum ROI from this groupby dataframe

max_roi_lan= roi_languages_df[roi_languages_df["ROI"]==roi_languages_df["ROI"].max()]
max_roi_lan
print(f'The language of the movie with highest average ROI is {max_roi_lan["original_language"].iloc[0]}')
# create the non-null data frame for genres

imdb_df_no_null_genre = imdb_df_new[~imdb_df_new["genres"].isna()]
imdb_df_no_null_genre.head()
genres_list = []

for index, rows in imdb_df_no_null_genre.iterrows():

  genre_lst = rows["genres"]

  for elem in genre_lst:

    genres_list.append(elem["name"])
    unique_genres= list(set(genres_list))
    print(unique_genres)

Streaming output truncated to the last 5000 lines.
['Documentary', 'Drama', 'Action', 'Adventure', 'Music', 'Horror', 'War', 'Animation', 'Crime', 'Science Fiction', 'History', 'Romance', 'Western', 'Fantasy', 'Comedy', 'Family', 'Foreign', 'Mystery', 'Thriller']
['Documentary', 'Drama', 'Action', 'Adventure', 'Music', 'Horror', 'War', 'Animation', 'Crime', 'Science Fiction', 'History', 'Romance', 'Western', 'Fantasy', 'Comedy', 'Family', 'Foreign', 'Mystery', 'Thriller']
['Documentary', 'Drama', 'Action', 'Adventure', 'Music', 'Horror', 'War', 'Animation', 'Crime', 'Science Fiction', 'History', 'Romance', 'Western', 'Fantasy', 'Comedy', 'Family', 'Foreign', 'Mystery', 'Thriller']
['Documentary', 'Drama', 'Action', 'Adventure', 'Music', 'Horror', 'War', 'Animation', 'Crime', 'Science Fiction', 'History', 'Romance', 'Western', 'Fantasy', 'Comedy', 'Family', 'Foreign', 'Mystery', 'Thriller']
['Documentary', 'Drama', 'Action', 'Adventure', 'Music', 'Horror', 'War', 'Animation', 'Crime', '

Make a table of all the producers and directors of each movie. Find the top 3 producers who have produced movies with the highest average RoI?

In [6]:
# creating non-null data frame for crew

imdb_df_no_null_crew = imdb_df_new[~imdb_df_new["crew"].isna()]
# creating a function to get all the producers and directors of movie

def get_prod_n_director(crew_list):
  

  try:

    prod_lst=[]
    dir_lst =[]

    for elem in crew_list:

      if elem["job"]=="Producer":
        prod_lst.append(elem["name"])

      elif elem["job"]=="Director" :
        dir_lst.append(elem["name"])


    return prod_lst,dir_lst

  except:
    return [],[]
    get_prod_n_director(imdb_df_no_null_crew.loc[1,"crew"])
    # applying these function on the entire data frame

imdb_df_new[["Producer","Director"]]= imdb_df_no_null_crew.apply(lambda row: get_prod_n_director(row["crew"]),axis=1,result_type="expand")
imdb_df_new.head()

,budget,genres,imdb_id,original_language,original_title,revenue,cast,crew,Profit,ROI,Producer,Director
0,14000000,"[{'id': 35, 'name': 'Comedy'}]",tt2637294,en,Hot Tub Time Machine 2,12314651,"[{'cast_id': 4, 'character': 'Lou', 'credit_id...","[{'credit_id': '59ac067c92514107af02c8c8', 'de...",-1685349,-0.120382,[Andrew Panay],[Steve Pink]
1,40000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",tt0368933,en,The Princess Diaries 2: Royal Engagement,95149435,"[{'cast_id': 1, 'character': 'Mia Thermopolis'...","[{'credit_id': '52fe43fe9251416c7502563d', 'de...",55149435,1.378736,"[Whitney Houston, Mario Iscovich, Debra Martin...",[Garry Marshall]
2,3300000,"[{'id': 18, 'name': 'Drama'}]",tt2582802,en,Whiplash,13092000,"[{'cast_id': 5, 'character': 'Andrew Neimann',...","[{'credit_id': '54d5356ec3a3683ba0000039', 'de...",9792000,2.967273,"[David Lancaster, Michel Litvak, Jason Blum, H...",[Damien Chazelle]
3,1200000,"[{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...",tt1821480,hi,Kahaani,16000000,"[{'cast_id': 1, 'character': 'Vidya Bagchi', '...","[{'credit_id': '52fe48779251416c9108d6eb', 'de...",14800000,12.333333,[Sujoy Ghosh],[Sujoy Ghosh]
4,16000000,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",tt1380152,ko,마린보이,3923970,"[{'cast_id': 3, 'character': 'Chun-soo', 'cred...","[{'credit_id': '52fe464b9251416c75073b43', 'de...",-12076030,-0.754752,[],[Jong-seok Yoon]
